In [1]:
## loading essential libraries
import numpy as np, os.path, pandas as pd, sys, matplotlib.pyplot as plt, copy
from pytorch import run_final_model, run_cross_valid, combined_plot, save_outputs, stat_new_data
from pytorch import feature_slist, feature_list, stat, specific, explore, print_optimal_tables, print_stat_tables, combined_plot_noise, individual_plot_noise, learning_curve, plot_learning_curve
from read_in_out import initiate_data, initiate_RNN_data, analysis_options, ML_analysis, compare_braced_input_data, compare_braced_output_data, introduce_marker_drop
from joblib import Parallel, delayed
from pytorch_utilities import transformer
feat_order = ['JA','JM','JRF']#,'MA','MF']
import seaborn as sns

## 17 April 
## unclear about windows of CNN etc etc ... although unsure what to do .. using 5 for everything right now
## for lc curve .. it's simple just run the lc code and then plot it .. 
## need to see the status for noise ... fix this combined_plot_noise .. top priority

## 9 april
## ensure to activate conda gamma
## some of the model have poor performance 
## unclear about windows of CNN etc etc ... although unsure what to do .. 

##18 nov ..
## basically the issue is that the unseen subject in subect exposed and naive are not same ..
## I have tried part of subject common ... give it a try that all subject are common
## but then issue might be that lack of data in subject exposed .. so carefully choose ...

##27 Dec ..
##update subject in super held-out ... this lead to some issues in CV .. will change later...

In [2]:
### some scripts need to compute
def initiate_naive_braced(dfo, dfn):
    dfo.naive_braced         = copy.deepcopy(dfn.naive)  
    dfo.naive_braced.arg     = copy.deepcopy(dfo.naive.arg)  ## creating an instance of naive_braced
    dfo.naive_braced.arch    = copy.deepcopy(dfo.naive.arch)  ## creating an instance of naive_braced
    dfo.naive_braced.save_name = copy.deepcopy(dfo.naive.save_name)  ## creating an instance of naive_braced
    dfo.naive_braced.subject = 'naive_braced'
    return dfo

def train_final_models(D):
    ## train final model with best-avg-validation accuracy
    for i in range(3):
        specific(D.exposed,i)
        specific(D.naive  ,i)
    return None

def compute_stat(f):
    for D in f:
        for i in range(3):
            D.exposed = stat(D.exposed,i)
            print(D.exposed.subject, 10*'--***--',i)
            D.naive   = stat(D.naive,i)
            print(D.naive.subject, 10*'--***--',i)
            try:
                D.exposed_unseen.subject = 'exposed_unseen'
                D.exposed_unseen = stat(D.exposed_unseen, i)
                print('exposed_unseen', 10*'--***--',i)
            except:
                None
            try:
                D.naive_braced = stat(D.naive_braced, i)
                print('naive_braced', 10*'--***--',i)
            except:
                None
            print(f"\n\n")
    return f

def avg_stat(fm):
    for f in fm:
        for j in [f.exposed, f.naive]:
            a,b = [],[]
            for i in f.feature:
                a.append(j.NRMSE[i])
                b.append(j.pc[i])
            a = pd.concat(a, axis=1)
            b = pd.concat(b, axis=1)
            # print('%',np.around(np.mean(a),2),np.around(np.std(a),2), j.kind, j.subject, 'NRMSE')
            # print('%',np.around(np.mean(b),2),np.around(np.std(b),2), j.kind, j.subject, 'pc')

# avg_stat([fm.LM])
from itertools import chain
def merge_lists(nested_list):
    return list(chain.from_iterable(nested_list))


def plot_final_results(df1, save_name, plot_what):
    fig, ax = plt.subplots()
    xtlabels = [t.save_name for t in df1]
    colors = ['#1f77b4', '#ff7f0e', '#2ca02c']*10  # blue, orange, green
    labels = df1[0].feature

    if plot_what == 'pc':
        df = [t.pc for t in df1]
        ax.set_ylabel('pearson correlation', size=12)    
        data = [data_dict[key].values.ravel().tolist() for data_dict in df for key in data_dict.keys()]
        box_width = 0.5
        pos = [i for sl in [np.arange(3)+4.5*i for i in range(len(data)//3)] for i in sl]
        bplot = ax.boxplot(data, positions=pos, patch_artist=True, whis=None, showfliers=False,medianprops=dict(color='black'))    
        for box, color in zip(bplot['boxes'], colors):
            box.set_facecolor(color)
        ax.set_ylim(-1.02,1.02)
        for patch, color in zip(bplot['boxes'], colors):
            patch.set_facecolor(color)
    elif plot_what == 'nparams':
        data = merge_lists([t.nparams for t in df1])
        ax.set_ylabel('# parameters', size=12)    
        pos = [i for sl in [np.arange(3)+4.5*i for i in range(len(data)//3)] for i in sl]
        ax.bar(pos, data, color = colors*20)
        ax.set_xlim(pos[0]-0.5, pos[-1]+0.5)
    ax.set_xticks(pos[1::3])
    ax.set_xticklabels(xtlabels,rotation=45)
    ax.tick_params(axis='both', labelsize=12)  
    ax.grid(axis='y', color='lightgray', linestyle='-', alpha=0.3)
    from matplotlib.patches import Patch
    legend_patches = [Patch(facecolor=c, label=l) for c, l in zip(colors, labels)]
    legend = ax.legend(handles=legend_patches,ncol=1,bbox_to_anchor=(0.92, 0.3), loc='center', )  # You can change the location as needed
    legend.get_frame().set_alpha(1) 

    plt.tight_layout()
    plt.savefig(f'./plots_out/{save_name}.{plot_what}.pdf', dpi=600)
    plt.show()
    plt.close()


In [3]:
# ### function to load_model_index
# def load_model_index(fm, fm_braced, data_kind):
#     if 'LM' in data_kind:
#         fm.LM.exposed.arg      = [43, 43, 43]
#         fm.LM.naive.arg        = [43, 43, 43]
#         fm.LM.exposed.arch, fm.LM.naive.arch  = ['LM']*3, ['LM']*3
#         fm.LM.exposed_unseen     = copy.deepcopy(fm.LM.exposed)
#         fm.LM.save_name, fm.LM.exposed.save_name, fm.LM.naive.save_name, fm.LM.exposed_unseen.save_name = ('LM',)*4
#         fm.LM = initiate_naive_braced(fm.LM, fm_braced.LM) ## creating an instance of naive_braced
        
#     if 'rf' in data_kind:
#         fm.rf.exposed.arg      = [1, 1, 1]
#         fm.rf.naive.arg        = [1, 1, 1]
#         fm.rf.exposed.arch, fm.rf.naive.arch = ['rf']*3, ['rf']*3
#         fm.rf.exposed_unseen     = copy.deepcopy(fm.rf.exposed)
#         fm.rf.save_name, fm.rf.exposed.save_name, fm.rf.naive.save_name, fm.rf.exposed_unseen.save_name = ('RF',)*4
#         fm.rf = initiate_naive_braced(fm.rf, fm_braced.rf) ## creating an instance of naive_braced

#     if 'transformer' in data_kind:
#         fm.transformer.exposed.arg      = [23,23,23]
#         fm.transformer.naive.arg        = [23,23,23]
#         fm.transformer.exposed.arch, fm.transformer.naive.arch = ['transformer']*3, ['transformer']*3
#         fm.transformer.exposed_unseen     = copy.deepcopy(fm.transformer.exposed)
#         fm.transformer.save_name, fm.transformer.exposed.save_name, fm.transformer.naive.save_name, fm.transformer.exposed_unseen.save_name = ('transformer',)*4
#         fm.transformer = initiate_naive_braced(fm.transformer, fm_braced.transformer) ## creating an instance of naive_braced

#     if 'CNN' in data_kind:
#         fm.CNN.exposed.arg        = [248,235, 52]
#         fm.CNN.naive.arg          = [52, 235, 215]  ##234
#         fm.CNN.exposed.arch, fm.CNN.naive.arch = ['CNN']*3, ['CNN']*3
#         fm.CNN.exposed_unseen     = copy.deepcopy(fm.CNN.exposed)
#         fm.CNN.save_name, fm.CNN.exposed.save_name, fm.CNN.naive.save_name, fm.CNN.exposed_unseen.save_name = ('CNN',)*4
#         fm.CNN = initiate_naive_braced(fm.CNN, fm_braced.CNN) ## creating an instance of naive_braced

    
#     if 'CNNLSTM' in data_kind:
#         fm.CNNLSTM.exposed.arg        = [387, 387, 21]
#         fm.CNNLSTM.naive.arg          = [387, 387, 391] ## 389
#         fm.CNNLSTM.exposed.arch, fm.CNNLSTM.naive.arch = ['CNNLSTM']*3, ['CNNLSTM']*3
#         fm.CNNLSTM.exposed_unseen     = copy.deepcopy(fm.CNNLSTM.exposed)
#         fm.CNNLSTM.save_name, fm.CNNLSTM.exposed.save_name, fm.CNNLSTM.naive.save_name, fm.CNNLSTM.exposed_unseen.save_name = ('CNNLSTM',)*4
#         fm.CNNLSTM = initiate_naive_braced(fm.CNNLSTM, fm_braced.CNNLSTM) ## creating an instance of naive_braced
        
#     if 'xgbr' in data_kind:
#         # fm.xgbr.exposed.arg        = [218, 218, 218]  ## prefinal
#         # fm.xgbr.naive.arg          = [218, 218, 218]
#         fm.xgbr.exposed.arg        = [26, 26, 96]
#         fm.xgbr.naive.arg          = [26, 26, 47]
#         fm.xgbr.exposed.arch, fm.xgbr.naive.arch = ['xgbr']*3, ['xgbr']*3
#         fm.xgbr.exposed_unseen     = copy.deepcopy(fm.xgbr.exposed)
#         fm.xgbr.save_name, fm.xgbr.exposed.save_name, fm.xgbr.naive.save_name, fm.xgbr.exposed_unseen.save_name = ('XGBR',)*4
#         fm.xgbr = initiate_naive_braced(fm.xgbr, fm_braced.xgbr) ## creating an instance of naive_braced

#     if 'NN' in data_kind:
#         fm.NN.exposed.arg        = [1043, 1043, 1043]
#         fm.NN.naive.arg          = [1043, 1091, 1043]
#         fm.NN.exposed.arch, fm.NN.naive.arch = ['NN']*3, ['NN']*3
#         fm.NN.exposed_unseen     = copy.deepcopy(fm.NN.exposed)
#         fm.NN.save_name, fm.NN.exposed.save_name, fm.NN.naive.save_name, fm.NN.exposed_unseen.save_name = ('FFNN',)*4
#         fm.NN = initiate_naive_braced(fm.NN, fm_braced.NN) ## creating an instance of naive_braced

#     if 'RNN' in data_kind:    
#         fm.VRNN = copy.deepcopy(fm.RNN) 
#         fm.LSTM = copy.deepcopy(fm.RNN) 
#         fm.GRU  = copy.deepcopy(fm.RNN) 
        
#         fm.VRNN.exposed.arg       = [3119, 3123, 3125] ## 3411  0:3123
#         fm.VRNN.naive.arg         = [3169, 2051, 100] 
#         fm.VRNN.exposed.arch, fm.VRNN.naive.arch = ['RNN']*3, ['RNN']*3   ## (SimpleRNN, LSTM, GRU)
#         fm.VRNN.exposed_unseen     = copy.deepcopy(fm.VRNN.exposed)
#         fm.VRNN.save_name, fm.VRNN.exposed.save_name, fm.VRNN.naive.save_name, fm.VRNN.exposed_unseen.save_name = ('VRNN',)*4
#         fm.VRNN = initiate_naive_braced(fm.VRNN, fm_braced.RNN) ## creating an instance of naive_braced
    
#         fm.LSTM.exposed.arg       = [3458,3738,3458] ## 3458
#         fm.LSTM.naive.arg         = [3458,3736,4579] #[7493,4108, 4396 ]
#         fm.LSTM.exposed.arch, fm.LSTM.naive.arch = ['RNN']*3, ['RNN']*3 
#         fm.LSTM.exposed_unseen     = copy.deepcopy(fm.LSTM.exposed)
#         fm.LSTM.save_name, fm.LSTM.exposed.save_name, fm.LSTM.naive.save_name, fm.LSTM.exposed_unseen.save_name = ('LSTM',)*4
#         fm.LSTM = initiate_naive_braced(fm.LSTM, fm_braced.RNN) ## creating an instance of naive_braced
    
#         fm.GRU.exposed.arg        = [9153, 10029, 10301]
#         fm.GRU.naive.arg          = [9845, 8309, 6916 ]
#         fm.GRU.exposed.arch, fm.GRU.naive.arch = ['RNN']*3, ['RNN']*3   ## (SimpleRNN, LSTM, GRU)
#         fm.GRU.exposed_unseen     = copy.deepcopy(fm.GRU.exposed)
#         fm.GRU.save_name, fm.GRU.exposed.save_name, fm.GRU.naive.save_name, fm.GRU.exposed_unseen.save_name = ('GRU',)*4
#         fm.GRU = initiate_naive_braced(fm.GRU, fm_braced.RNN) ## creating an instance of naive_braced

#     if 'convLSTM' in data_kind:
#         # fm.convLSTM.exposed.arg        = [26, 51, 31]  
#         # fm.convLSTM.naive.arg          = [36, 51, 36]
#         fm.convLSTM.exposed.arg        = [26, 51, 31]  
#         fm.convLSTM.naive.arg          = [36, 252, 254] ### 254 for JRF, 63 for JA
#         fm.convLSTM.exposed.arch, fm.convLSTM.naive.arch = ['convLSTM']*3, ['convLSTM']*3
#         fm.convLSTM.exposed_unseen     = copy.deepcopy(fm.convLSTM.exposed)
#         fm.convLSTM.save_name, fm.convLSTM.exposed.save_name, fm.convLSTM.naive.save_name, fm.convLSTM.exposed_unseen.save_name = ('convLSTM',)*4
#         fm.convLSTM = initiate_naive_braced(fm.convLSTM, fm_braced.convLSTM) ## creating an instance of naive_braced
    
#     return fm    

In [4]:
### function to load_model_index
def load_model_index(fm, fm_braced, data_kind):
    if 'LM' in data_kind:
        fm.LM.exposed.arg      = [43, 43, 43]
        fm.LM.naive.arg        = [43, 43, 43]
        fm.LM.exposed.arch, fm.LM.naive.arch  = ['LM']*3, ['LM']*3
        fm.LM.exposed_unseen     = copy.deepcopy(fm.LM.exposed)
        fm.LM.save_name, fm.LM.exposed.save_name, fm.LM.naive.save_name, fm.LM.exposed_unseen.save_name = ('LM',)*4
        fm.LM = initiate_naive_braced(fm.LM, fm_braced.LM) ## creating an instance of naive_braced
        
    if 'rf' in data_kind:
        fm.rf.exposed.arg      = [1, 5, 1]
        fm.rf.naive.arg        = [1, 1, 1]
        fm.rf.exposed.arch, fm.rf.naive.arch = ['rf']*3, ['rf']*3
        fm.rf.exposed_unseen     = copy.deepcopy(fm.rf.exposed)
        fm.rf.save_name, fm.rf.exposed.save_name, fm.rf.naive.save_name, fm.rf.exposed_unseen.save_name = ('RF',)*4
        fm.rf = initiate_naive_braced(fm.rf, fm_braced.rf) ## creating an instance of naive_braced

    if 'transformer' in data_kind:
        fm.transformer.exposed.arg      = [23,23,23]
        fm.transformer.naive.arg        = [23,23,23]
        fm.transformer.exposed.arch, fm.transformer.naive.arch = ['transformer']*3, ['transformer']*3
        fm.transformer.exposed_unseen     = copy.deepcopy(fm.transformer.exposed)
        fm.transformer.save_name, fm.transformer.exposed.save_name, fm.transformer.naive.save_name, fm.transformer.exposed_unseen.save_name = ('transformer',)*4
        fm.transformer = initiate_naive_braced(fm.transformer, fm_braced.transformer) ## creating an instance of naive_braced
            
    if 'xgbr' in data_kind:
        # fm.xgbr.exposed.arg        = [218, 218, 218]  ## prefinal
        # fm.xgbr.naive.arg          = [218, 218, 218]
        fm.xgbr.exposed.arg        = [26, 26, 96]
        fm.xgbr.naive.arg          = [26, 26, 47]
        fm.xgbr.exposed.arch, fm.xgbr.naive.arch = ['xgbr']*3, ['xgbr']*3
        fm.xgbr.exposed_unseen     = copy.deepcopy(fm.xgbr.exposed)
        fm.xgbr.save_name, fm.xgbr.exposed.save_name, fm.xgbr.naive.save_name, fm.xgbr.exposed_unseen.save_name = ('XGBR',)*4
        fm.xgbr = initiate_naive_braced(fm.xgbr, fm_braced.xgbr) ## creating an instance of naive_braced

    if 'NN' in data_kind:
        fm.NN.exposed.arg        = [1043, 1043, 1043]
        fm.NN.naive.arg          = [1043, 1091, 1043]
        fm.NN.exposed.arch, fm.NN.naive.arch = ['NN']*3, ['NN']*3
        fm.NN.exposed_unseen     = copy.deepcopy(fm.NN.exposed)
        fm.NN.save_name, fm.NN.exposed.save_name, fm.NN.naive.save_name, fm.NN.exposed_unseen.save_name = ('FFNN',)*4
        fm.NN = initiate_naive_braced(fm.NN, fm_braced.NN) ## creating an instance of naive_braced

    if 'convLSTM' in data_kind:
        fm.convLSTM.exposed.arg        = [26, 51, 31]  
        fm.convLSTM.naive.arg          = [36,  33, 26] ### 254 for JRF, 63 for JA
        fm.convLSTM.exposed.arch, fm.convLSTM.naive.arch = ['convLSTM']*3, ['convLSTM']*3
        fm.convLSTM.exposed_unseen     = copy.deepcopy(fm.convLSTM.exposed)
        fm.convLSTM.save_name, fm.convLSTM.exposed.save_name, fm.convLSTM.naive.save_name, fm.convLSTM.exposed_unseen.save_name = ('convLSTM',)*4
        fm.convLSTM = initiate_naive_braced(fm.convLSTM, fm_braced.convLSTM) ## creating an instance of naive_braced

    if 'CNN' in data_kind:
        fm.CNN.exposed.arg        = [28, 51, 31]  ### 26, 51, 31
        fm.CNN.naive.arg          = [36,  19, 26] ### 254 for JRF, 63 for JA
        fm.CNN.exposed.arch, fm.CNN.naive.arch = ['CNN']*3, ['CNN']*3
        fm.CNN.exposed_unseen     = copy.deepcopy(fm.CNN.exposed)
        fm.CNN.save_name, fm.CNN.exposed.save_name, fm.CNN.naive.save_name, fm.CNN.exposed_unseen.save_name = ('CNN',)*4
        fm.CNN = initiate_naive_braced(fm.CNN, fm_braced.CNN) ## creating an instance of naive_braced

    if 'CNNLSTM' in data_kind:
        fm.CNNLSTM.exposed.arg        = [387, 387, 23]
        fm.CNNLSTM.naive.arg          = [387, 387, 391] ## 389
        fm.CNNLSTM.exposed.arch, fm.CNNLSTM.naive.arch = ['CNNLSTM']*3, ['CNNLSTM']*3
        fm.CNNLSTM.exposed_unseen     = copy.deepcopy(fm.CNNLSTM.exposed)
        fm.CNNLSTM.save_name, fm.CNNLSTM.exposed.save_name, fm.CNNLSTM.naive.save_name, fm.CNNLSTM.exposed_unseen.save_name = ('CNNLSTM',)*4
        fm.CNNLSTM = initiate_naive_braced(fm.CNNLSTM, fm_braced.CNNLSTM) ## creating an instance of naive_braced
    
    if 'RNN' in data_kind:    
        fm.VRNN = copy.deepcopy(fm.RNN) 
        fm.LSTM = copy.deepcopy(fm.RNN) 
        fm.GRU  = copy.deepcopy(fm.RNN) 
        
        fm.VRNN.exposed.arg       = [3119, 3411, 3125] ## 3123
        fm.VRNN.naive.arg         = [3169, 2051, 3411]  #100
        fm.VRNN.exposed.arch, fm.VRNN.naive.arch = ['RNN']*3, ['RNN']*3   ## (SimpleRNN, LSTM, GRU)
        fm.VRNN.exposed_unseen     = copy.deepcopy(fm.VRNN.exposed)
        fm.VRNN.save_name, fm.VRNN.exposed.save_name, fm.VRNN.naive.save_name, fm.VRNN.exposed_unseen.save_name = ('VRNN',)*4
        fm.VRNN = initiate_naive_braced(fm.VRNN, fm_braced.RNN) ## creating an instance of naive_braced
    
        fm.LSTM.exposed.arg       = [3458,3738,3458] ## 3458
        fm.LSTM.naive.arg         = [3458,3736,4579] #[7493,4108, 4396 ]
        fm.LSTM.exposed.arch, fm.LSTM.naive.arch = ['RNN']*3, ['RNN']*3 
        fm.LSTM.exposed_unseen     = copy.deepcopy(fm.LSTM.exposed)
        fm.LSTM.save_name, fm.LSTM.exposed.save_name, fm.LSTM.naive.save_name, fm.LSTM.exposed_unseen.save_name = ('LSTM',)*4
        fm.LSTM = initiate_naive_braced(fm.LSTM, fm_braced.RNN) ## creating an instance of naive_braced
    
        fm.GRU.exposed.arg        = [9153, 10029, 10301]
        fm.GRU.naive.arg          = [9845, 8309, 6916 ]
        fm.GRU.exposed.arch, fm.GRU.naive.arch = ['RNN']*3, ['RNN']*3   ## (SimpleRNN, LSTM, GRU)
        fm.GRU.exposed_unseen     = copy.deepcopy(fm.GRU.exposed)
        fm.GRU.save_name, fm.GRU.exposed.save_name, fm.GRU.naive.save_name, fm.GRU.exposed_unseen.save_name = ('GRU',)*4
        fm.GRU = initiate_naive_braced(fm.GRU, fm_braced.RNN) ## creating an instance of naive_braced

    return fm    

In [5]:
# ### convLSTM , CNN,  CNNLSTM, 
# ##   


fm = ML_analysis('final_model_list', ['RNN'], 5)
fmb = ML_analysis('final_model_list', ['RNN'], 5, add1 = 'Braced_')
fm = load_model_index(fm, fmb, ['RNN'])


D = fm.LSTM
# for i in np.arange(1,3):
#     specific(D.exposed,i)
#     specific(D.naive, i)
# # specific(D.naive,2)

# print('--***--'*15)
# compute_stat([D])
# print(D.exposed.pc['JA'].mean().mean(),        D.exposed.pc['JM'].mean().mean(),        D.exposed.pc['JRF'].mean().mean())
# print(D.naive.pc['JA'].mean().mean(),          D.naive.pc['JM'].mean().mean(),          D.naive.pc['JRF'].mean().mean())
# print(D.exposed_unseen.pc['JA'].mean().mean(), D.exposed_unseen.pc['JM'].mean().mean(), D.exposed_unseen.pc['JRF'].mean().mean())
sys.exit()

Loading Braced data .. for normal data use empty string as input ...


SystemExit: 

/Users/kpxp057/anaconda3/envs/gamma/lib/python3.12/site-packages/IPython/core/interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [60]:
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Input, Conv1D, MaxPooling2D, UpSampling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam


def introduce_marker_drop(data_in, seed, prob_of_missing=0.05, number_of_miss=1):
    np.random.seed(seed=seed)  ##use seed as 25, 12, 1992
    markers = data_in.columns.shape[0]//3      ## 19 markers in the input data
    data = data_in.copy()
    for i in data.index:
        if np.random.binomial(1, prob_of_missing):
            ## how many marker to miss
            marker_index = np.random.choice(markers, number_of_miss, replace=False)
            for m in marker_index:
                marker_ind = np.arange(3*m,3*m+3)
                data.loc[i, marker_ind] = [-1]
    np.random.seed(seed=None)
    ##a2[a2.isna().any(axis=1)]
    return data
# df_inp = pd.concat([fm.LSTM.data.i1.T1, fm.LSTM.data.i1.T2, fm.LSTM.data.i1.T3])


def introduce_marker_drop_window(data_in, seed, prob_of_missing=0.05, number_of_miss=1):
    np.random.seed(seed=seed)  ##use seed as 25, 12, 1992
    markers = 19 ## 19 markers in the input data
    data = data_in.copy()
    for i in np.arange(data.shape[0]):
        if np.random.binomial(1, prob_of_missing):
            ## how many marker to miss
            marker_index = np.random.choice(markers, number_of_miss, replace=False)
            for m in marker_index:
                df_inp[1, 4, 3*m:3*m+3] = [-1,-1,1]
    np.random.seed(seed=None)
    return data


output_array = np.concatenate((fm.LSTM.data.i1.T1, fm.LSTM.data.i1.T2, fm.LSTM.data.i1.T3), axis=0)
input_array = introduce_marker_drop_window(output_array,25)

output_array_test = np.concatenate((fm.LSTM.data.i2.T1, fm.LSTM.data.i2.T2, fm.LSTM.data.i2.T3), axis=0)
input_array_test = introduce_marker_drop_window(output_array_test,25)


import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Reshape, LayerNormalization, MultiHeadAttention, Dropout, Add
from tensorflow.keras.models import Model

# Define a single Transformer Encoder layer
def transformer_encoder(input_tensor, num_heads, d_model, dropout_rate=0.1):
    # Multi-Head Self-Attention
    attention_output = MultiHeadAttention(num_heads=num_heads, key_dim=d_model)(input_tensor, input_tensor)
    # Add & Normalize
    attention_output = Add()([input_tensor, attention_output])
    attention_output = LayerNormalization(epsilon=1e-6)(attention_output)
    # Feed-Forward Network
    ff_output = Dense(d_model * 4, activation="relu")(attention_output)
    ff_output = Dense(d_model)(ff_output)
    # Add & Normalize
    ff_output = Add()([attention_output, ff_output])
    ff_output = LayerNormalization(epsilon=1e-6)(ff_output)
    # Dropout
    ff_output = Dropout(dropout_rate)(ff_output)
    return ff_output



from tensorflow.keras.layers import Dense, Add, Dropout, LayerNormalization

from tensorflow.keras.layers import ConvLSTM2D, BatchNormalization, Dropout
from tensorflow.keras.layers import Dense, LayerNormalization, Add

from tensorflow.keras.layers import ConvLSTM2D, BatchNormalization, Dense, Add, Dropout, GlobalAveragePooling2D

def convlstm_encoder(input_tensor, filters=64, kernel_size=(3, 3), dropout_rate=0.1):
    # ConvLSTM2D layer
    convlstm_output = ConvLSTM2D(
        filters=filters,
        kernel_size=kernel_size,
        padding="same",
        return_sequences=False,  # Set to True if stacking multiple ConvLSTM layers
        activation="tanh"
    )(input_tensor)
    
    # Batch normalization
    normalized_output = BatchNormalization()(convlstm_output)
    
    # Global pooling to ensure compatibility with Dense layer
    pooled_output = GlobalAveragePooling2D()(normalized_output)
    
    # Dense layer
    dense_output = Dense(filters, activation="relu")(pooled_output)
    
    # Residual connection
    residual_output = Add()([pooled_output, dense_output])
    
    # Dropout
    final_output = Dropout(dropout_rate)(residual_output)
    
    return final_output

# Input Shape: (5, 58) where 5 is the timestep and 58 is the feature dimension
input_timesteps = 5
input_dim = 58  # Feature dimension
projected_dim = 58  # Desired feature dimension after projection


from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model

input_tensor = Input(shape=(5, 58, 58, 1))  # Example shape: (time_steps, height, width, channels)
output_tensor = convlstm_encoder(input_tensor)

model = Model(inputs=input_tensor, outputs=output_tensor)




In [34]:
df_inp.shape

(1184, 5, 58)

In [31]:
df_inp[1,0]

array([ 2.02451508e+02,  8.55659485e+02,  1.23236047e+03,  1.99971863e+02,
        9.62214905e+02,  9.97198792e+02,  2.32073502e+02,  7.23145935e+02,
        1.13282300e+03,  2.25652771e+02,  6.98975525e+02,  9.78988464e+02,
        7.71879730e+01,  9.30029419e+02,  1.09962781e+03,  3.62066071e+02,
        8.39636841e+02,  1.21184021e+03,  4.37830849e+01,  7.67830933e+02,
        1.20295032e+03, -3.59445229e+01,  7.22577942e+02,  1.03494958e+03,
       -3.21442947e+01,  6.21139099e+02,  8.85662903e+02,  1.57610626e+01,
        4.74860657e+02,  9.07690002e+02,  1.04572044e+02,  3.79749756e+02,
        9.49484924e+02,  8.14501648e+01,  3.62141266e+02,  8.98948120e+02,
        8.60955734e+01,  2.86909546e+02,  9.74585632e+02,  4.56463699e+01,
        7.83607605e+02,  7.73993347e+02,  2.27530396e+02,  9.83945801e+02,
        7.67492676e+02,  1.46655869e+02,  9.85666016e+02,  7.70863708e+02,
        1.14012222e+02,  3.32908356e+02,  1.00710242e+03,  1.48931671e+02,
        2.50902390e+02,  

In [11]:
fm.LSTM.data.i1.T1.shape

(351, 5, 58)